In [ ]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import sqlite3

class BookStoreApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistem Pembelian Buku")
        
        # Initialize database
        self.conn = sqlite3.connect('user_data.db')
        self.cursor = self.conn.cursor()
        self.create_table()

        self.login_page()

    def create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                username TEXT PRIMARY KEY,
                password TEXT NOT NULL
            )
        ''')
        self.conn.commit()

    def login_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Login").pack(pady=10)
        tk.Label(self.frame, text="Username").pack()
        self.username_entry = tk.Entry(self.frame)
        self.username_entry.pack()
        tk.Label(self.frame, text="Password").pack()
        self.password_entry = tk.Entry(self.frame, show='*')
        self.password_entry.pack()
        tk.Button(self.frame, text="Login", command=self.authenticate).pack(pady=10)
        tk.Button(self.frame, text="Register", command=self.register_page).pack(pady=5)

    def register_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Registrasi").pack(pady=10)
        tk.Label(self.frame, text="Username").pack()
        self.reg_username_entry = tk.Entry(self.frame)
        self.reg_username_entry.pack()
        tk.Label(self.frame, text="Password").pack()
        self.reg_password_entry = tk.Entry(self.frame, show='*')
        self.reg_password_entry.pack()
        tk.Button(self.frame, text="Register", command=self.save_registration).pack(pady=10)
        tk.Button(self.frame, text="Kembali", command=self.login_page).pack(pady=5)

    def save_registration(self):
        username = self.reg_username_entry.get()
        password = self.reg_password_entry.get()
        
        if username and password:
            try:
                self.cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
                self.conn.commit()
                messagebox.showinfo("Registrasi", "Registrasi berhasil!")
                self.login_page()
            except sqlite3.IntegrityError:
                messagebox.showerror("Registrasi", "Username sudah terdaftar!")
        else:
            messagebox.showerror("Registrasi", "Username dan Password tidak boleh kosong!")

    def authenticate(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        self.cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
        user = self.cursor.fetchone()

        if user:
            messagebox.showinfo("Login", "Login berhasil!")
            self.main_menu()
        else:
            messagebox.showerror("Login", "Username atau Password salah atau belum terdaftar!")

    def main_menu(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Menu Utama").pack(pady=10)
        tk.Button(self.frame, text="Pilih Produk", command=self.select_product).pack(pady=5)
        tk.Button(self.frame, text="Keluar", command=self.root.quit).pack(pady=5)

    def select_product(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Pilih Produk").pack(pady=10)

        products = [
            {"name": "Produk 1", "image": "product1.jpg"},
            {"name": "Produk 2", "image": "product2.jpg"},
            {"name": "Produk 3", "image": "product3.jpg"},
        ]

        for product in products:
            self.add_product(product)

        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def add_product(self, product):
        frame = tk.Frame(self.frame)
        frame.pack(pady=10)

        image = Image.open(product["image"])
        image = image.resize((100, 100), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image)

        label = tk.Label(frame, image=photo)
        label.image = photo
        label.pack(side="left")

        button = tk.Button(frame, text=f"Pilih {product['name']}", command=lambda: self.select_quantity(product))
        button.pack(side="right")

    def select_quantity(self, product):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text=f"Pilih Jumlah untuk {product['name']}").pack(pady=10)
        tk.Label(self.frame, text="Masukkan jumlah produk:").pack()
        self.quantity_entry = tk.Entry(self.frame)
        self.quantity_entry.pack()
        tk.Button(self.frame, text="Tambah ke Keranjang", command=lambda: self.add_to_cart(product)).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Pemilihan Produk", command=self.select_product).pack(pady=5)

    def add_to_cart(self, product):
        quantity = self.quantity_entry.get()
        if quantity.isdigit() and int(quantity) > 0:
            self.cart = {"product": product["name"], "quantity": int(quantity)}
            messagebox.showinfo("Keranjang", f"{quantity} {product['name']} ditambahkan ke keranjang!")
            self.checkout()
        else:
            messagebox.showerror("Keranjang", "Masukkan jumlah yang valid!")

    def checkout(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Checkout").pack(pady=10)
        tk.Label(self.frame, text=f"Produk: {self.cart['product']}").pack()
        tk.Label(self.frame, text=f"Jumlah: {self.cart['quantity']}").pack()
        tk.Label(self.frame, text="Metode Pembayaran:").pack()
        self.payment_method = tk.Entry(self.frame)
        self.payment_method.pack()
        tk.Button(self.frame, text="Konfirmasi Pembayaran", command=self.confirm_payment).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def confirm_payment(self):
        payment = self.payment_method.get()
        if payment:
            self.transaction_data = {"product": self.cart["product"], "quantity": self.cart["quantity"], "payment": payment}
            messagebox.showinfo("Pembayaran", "Pembayaran berhasil dikonfirmasi!")
            self.print_invoice()
        else:
            messagebox.showerror("Pembayaran", "Masukkan metode pembayaran!")

    def print_invoice(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Invoice").pack(pady=10)
        invoice_text = f"Produk: {self.transaction_data['product']}\nJumlah: {self.transaction_data['quantity']}\nMetode Pembayaran: {self.transaction_data['payment']}"
        tk.Label(self.frame, text=invoice_text).pack()
        tk.Button(self.frame, text="Transaksi Baru", command=self.main_menu).pack(pady=10)
        tk.Button(self.frame, text="Keluar", command=self.root.quit).pack(pady=5)

    def clear_frame(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = BookStoreApp(root)
    root.mainloop()

